In [ ]:
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Define custom sequences with slight length variations.
# For this example, we manually pad the sequences with gaps to simulate an alignment.
seq_records = [
    SeqRecord(Seq("ACDEFGHIKL"), id="seq1"),   # length 10
    SeqRecord(Seq("ACDEFGHIKL"), id="seq2"),   # length 10
    SeqRecord(Seq("ACDEFGHIK"), id="seq3"),    # length 9 (missing last residue)
    SeqRecord(Seq("ACDEFGHIKLM"), id="seq4")   # length 11 (one extra residue)
]

# Manually create an alignment by padding shorter sequences with gaps.
# Here, we choose an alignment length of 11 (the maximum length):
aligned_records = MultipleSeqAlignment([
    SeqRecord(Seq("ACDEFGHIKL-"), id="seq1"),
    SeqRecord(Seq("ACDEFGHIKL-"), id="seq2"),
    SeqRecord(Seq("ACDEFGHIK--"), id="seq3"),
    SeqRecord(Seq("ACDEFGHIKLM"), id="seq4")
])

print("Multiple Sequence Alignment:")
for record in aligned_records:
    print(f"{record.id}: {record.seq}")

# Identify conserved columns:
# A column is considered conserved if all non-gap characters in that column are the same.
alignment_length = aligned_records.get_alignment_length()
conserved_positions = []
for i in range(alignment_length):
    column = aligned_records[:, i]
    # Remove gap characters
    letters = [char for char in column if char != '-']
    if letters and all(char == letters[0] for char in letters):
        conserved_positions.append(i)

print("\nConserved column positions (0-indexed):", conserved_positions)

# Extract the conserved motif from one of the sequences (e.g., seq1)
motif = "".join(aligned_records[0].seq[i] for i in conserved_positions)
print("\nExtracted conserved motif from seq1:", motif)


In [ ]:
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Example MSA with slight length variations; padded with gaps for consistency.
aligned_records = MultipleSeqAlignment([
    SeqRecord(Seq("ACDEFGHIKL-"), id="seq1"),  # length 11
    SeqRecord(Seq("ACDEFGHIKL-"), id="seq2"),
    SeqRecord(Seq("ACDEFGHIK--"), id="seq3"),
    SeqRecord(Seq("ACDEFGHIKLM"), id="seq4")
])

# Let’s say our MSA analysis showed that columns 0, 1, 2, 3, 4, 7, and 8 are conserved.
conserved_positions = [0, 1, 2, 3, 4, 7, 8]

# Extract and print the conserved fragment (motif) from each sequence.
for record in aligned_records:
    fragment = "".join(record.seq[i] for i in conserved_positions)
    print(f"{record.id} fragment: {fragment}")


In [1]:
import os

# Get current working directory
current_dir = os.getcwd()

# Go one directory up
parent_dir = os.path.dirname(current_dir)

# Change working directory
os.chdir(parent_dir)

print("Now in:", os.getcwd())


Now in: /home/bharath-sooryaa-m/Documents/BIO/proj/Protein-Designing-With-Agents


In [2]:
from tools.uniprot_extended_tool import toolset 
dir = r"/home/bharath-sooryaa-m/Documents/BIO/proj/Protein-Designing-With-Agents/cache/session_20250406-154328_bfec90a8"
ntoolset = toolset(dir)
prot = ntoolset.get_all_sequence()
print(prot.items())

dict_items([('Q3KNS6', ['May be involved in transcriptional regulation']), ('Q6P2D0', ['May be involved in transcriptional regulation']), ('P17027', ['May be involved in transcriptional regulation. May have a role in embryonic development']), ('Q6ZN57', ['Probable transcription factor involved in neuronal differentiation and/or phenotypic maintenance']), ('Q9NX65', ['May be involved in transcriptional regulation']), ('P49910', ['May be involved in transcriptional regulation']), ('Q8N0Y2', ["Transcriptional regulator. Binds to the 5'-flanking critical region of the SCARF1 promoter"]), ('P17040', ['May be involved in transcriptional regulation']), ('P17021', ['May be involved in transcriptional regulation']), ('Q7L2R6', ['May be involved in transcriptional regulation']), ('P17022', ['May be involved in transcriptional regulation']), ('Q9ULM2', ['May be involved in transcriptional regulation']), ('O14628', ['May be involved in transcriptional regulation']), ('Q3KQV3', ['May be involved in

In [5]:
print(prot)  # Print the sequence of each protein record

{'Q3KNS6.json': [], 'Q6P2D0.json': [], 'P17027.json': [], 'Q6ZN57.json': [], 'Q9NX65.json': [], 'P49910.json': [], 'Q8N0Y2.json': [], 'P17040.json': [], 'P17021.json': [], 'Q7L2R6.json': [], 'P17022.json': [], 'Q9ULM2.json': [], 'O14628.json': [], 'Q3KQV3.json': [], 'P07199.json': [], 'Q96BR9.json': [], 'Q3MJ62.json': [], 'Q9H4T2.json': [], 'Q9Y222.json': [], 'Q6NX45.json': []}


In [ ]:
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB import PDBParser, Select, PDBIO

def perform_msa_and_extract_motif(seq_records, target_seq_id):
    """
    Given a list of SeqRecord objects (with sequences already padded to the same length to simulate an MSA),
    this function identifies conserved columns (ignoring gaps), and extracts the motif fragment from the 
    target sequence (specified by target_seq_id).
    
    Returns:
        motif_str: The extracted motif string.
        motif_alignment_indices: A list of alignment indices (0-indexed) where the motif is conserved.
        target_residue_numbers: The corresponding residue numbers in the target sequence,
                                assuming that positions with a gap ('-') are skipped.
    """
    # Create an MSA; here we assume the sequences are already padded with gaps
    alignment = MultipleSeqAlignment(seq_records)
    alignment_length = alignment.get_alignment_length()
    
    conserved_positions = []
    for i in range(alignment_length):
        column = alignment[:, i]
        # Remove gaps
        letters = [char for char in column if char != '-']
        if letters and all(char == letters[0] for char in letters):
            conserved_positions.append(i)
    
    # Find the target sequence record by its id
    target_record = next((rec for rec in alignment if rec.id == target_seq_id), None)
    if target_record is None:
        raise ValueError(f"Target sequence ID '{target_seq_id}' not found in alignment.")
    
    # Extract motif string from the target sequence using conserved positions.
    # Also build a mapping from alignment index to the residue number in the target.
    # Here, we assume that the target sequence's residue numbering is consecutive and starts at 1,
    # and that gaps are not counted.
    motif_chars = []
    target_res_nums = []
    current_res_num = 1  # starting residue number
    for i, char in enumerate(str(target_record.seq)):
        if char != '-':
            # This position corresponds to a real residue.
            if i in conserved_positions:
                motif_chars.append(char)
                target_res_nums.append(current_res_num)
            current_res_num += 1
    motif_str = "".join(motif_chars)
    
    return motif_str, conserved_positions, target_res_nums

def extract_fragment_from_pdb(pdb_filename, chain_id, res_num_list, output_filename):
    """
    Extracts a fragment from a PDB file based on a list of residue numbers from a given chain.
    The fragment is defined from the minimum to maximum residue number in res_num_list.
    
    Parameters:
      pdb_filename (str): Input PDB file path.
      chain_id (str): The chain identifier in the PDB.
      res_num_list (list of int): List of residue numbers corresponding to the conserved motif.
      output_filename (str): Output PDB file to save the fragment.
    """
    if not res_num_list:
        raise ValueError("No residue numbers provided for extraction.")
    
    start_res = min(res_num_list)
    end_res = max(res_num_list)
    
    class FragmentSelect(Select):
        def __init__(self, chain_id, start, end):
            self.chain_id = chain_id
            self.start = start
            self.end = end

        def accept_residue(self, residue):
            # Ensure the residue is from the target chain and within the range.
            if residue.get_parent().id == self.chain_id:
                # residue id is typically a tuple: (hetero flag, resseq, insertion code)
                resseq = residue.get_id()[1]
                if self.start <= resseq <= self.end:
                    return True
            return False

    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("target_structure", pdb_filename)
    io = PDBIO()
    io.set_structure(structure)
    io.save(output_filename, FragmentSelect(chain_id, start_res, end_res))
    print(f"Fragment (residues {start_res}-{end_res}) from chain {chain_id} extracted and saved to {output_filename}.")

# ----- Main workflow -----

# Step 1: Define custom sequences (simulate an MSA).
# Sequences are of roughly length ~10. We pad them manually with '-' to ensure they have the same length.

seq_records = [
    SeqRecord(Seq(prot[0][0]), id="seq1"),   # length 10
    SeqRecord(Seq(prot[1][0]), id="seq2"),   # length 10
    SeqRecord(Seq(prot[2][0]), id="seq3"),    # length 9; missing last residue
    SeqRecord(Seq(prot[3][0]), id="seq4")    # length 11; one extra residue
]


# Manually pad sequences to the maximum length
aligned_seq_records = []
max_len = max(len(rec.seq) for rec in seq_records)
for rec in seq_records:
    padded_seq = Seq(str(rec.seq).ljust(max_len, "-"))
    aligned_seq_records.append(SeqRecord(padded_seq, id=rec.id))


print("Multiple Sequence Alignment (simulated):")
for rec in aligned_seq_records:
    print(f"{rec.id}: {rec.seq}")

# Step 2: Identify conserved positions and extract the motif from a target sequence (e.g., seq1).
motif, conserved_pos, target_res_nums = perform_msa_and_extract_motif(aligned_seq_records, target_seq_id="seq1")
print("\nConserved alignment columns:", conserved_pos)
print("Extracted motif from seq1:", motif)
print("Mapped residue numbers in seq1 for conserved columns:", target_res_nums)

# For this example, assume that the conserved fragment in seq1 corresponds directly to residues 3 to 7.
# In practice, the mapping depends on the gap pattern; here, we simulate that the motif spans residues 3 to 7.
# (For demonstration, we choose the minimum and maximum from the target_res_nums corresponding to conserved positions.)
if target_res_nums:
    motif_residues = [target_res_nums[i] for i, pos in enumerate(range(len(str(aligned_seq_records[0].seq))) )
                      if pos in conserved_pos]
    print("Motif residue numbers in seq1:", motif_residues)
else:
    motif_residues = []

# Step 3: Extract the structural fragment from a PDB file.
# Assume we have a PDB file "input.pdb" where chain "A" corresponds to seq1.
# For simplicity, we'll extract the fragment spanning from the smallest to largest residue number in our motif.
if motif_residues:
    pdb_input = "cache/session_20250406-112148_1d5326ba/pdb/O14628.pdb"          # Replace with your actual PDB file
    chain_id = "A"                   # Chain corresponding to seq1 in the structure
    # For this example, we use the min and max residue numbers from our motif.
    start_res = min(motif_residues)
    end_res = max(motif_residues)
    output_pdb = "fragment_A_motif.pdb"
    extract_fragment_from_pdb(pdb_input, chain_id, motif_residues, output_pdb)
else:
    print("No motif residues identified; skipping PDB fragment extraction.")


In [ ]:
import py3Dmol

# Read the original PDB file
with open(pdb_input, 'r') as pdb_file:
    pdb_data = pdb_file.read()

# Create a viewer for the original structure
view = py3Dmol.view(width=800, height=600)
view.addModel(pdb_data, "pdb")

# Set a base style for the full structure (e.g., cartoon in light grey)
view.setStyle({'cartoon': {'color': 'lightgrey'}})

# Highlight the fragment corresponding to the motif:
# Build a list of residue numbers between start_res and end_res.
fragment_residues = list(range(start_res, end_res + 1))
# Apply a different style (e.g., cartoon in red) for the fragment
view.setStyle({'chain': chain_id, 'resi': fragment_residues}, {'cartoon': {'color': 'red'}})

view.zoomTo()
view.show()


In [5]:
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
import shutil
import sys

# Define input and output file paths
input_file = r"tools/input.fasta"
output_file = "output.fasta"

# Check if muscle is installed and in the system's PATH
if not shutil.which("muscle"):
    sys.exit("MUSCLE executable not found. Please install MUSCLE or add it to your PATH.")

try:
    # Create MuscleCommandline object (use `out`, not `output`)
    muscle_cline = MuscleCommandline(input=input_file, out=output_file)
    print(f"Running command:\n{muscle_cline}\n")

    # Execute MUSCLE
    stdout, stderr = muscle_cline()

    # Check for errors during MUSCLE execution
    if stderr:
        print(f"Error during MUSCLE execution:\n{stderr}")
    else:
        # Print success message
        print(f"MUSCLE alignment complete. Output saved to {output_file}")

        # Read and print the alignment
        alignment = AlignIO.read(output_file, "fasta")
        print(alignment)

except Exception as e:
    print(f"An error occurred: {e}")


SystemExit: MUSCLE executable not found. Please install MUSCLE or add it to your PATH.

C:\Users\BHARATH SOORYAA M\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
